In [5]:
import csv
import requests
import datetime

"""
A simple program to print the result of a Prometheus query as CSV.
"""
def query_csv(q, comment):
    # if len(sys.argv) != 3:
    #     print('Usage: {0} http://prometheus:9090 a_query'.format(sys.argv[0]))
    #     sys.exit(1)

    response = requests.get('http://localhost:9090/api/v1/query',
            params={'query': q})
    results = response.json()['data']['result']

    # Build a list of all labelnames used.
    labelnames = set()
    for result in results:
        labelnames.update(result['metric'].keys())

    # Canonicalize
    labelnames = sorted(labelnames)

    with open('test.csv', mode='a') as f:
        writer = csv.writer(f)
        # Write the header,
        writer.writerow(['time:', str(datetime.datetime.now()), 'comment:',comment])
        writer.writerow(['name', 'timestamp', 'value'] + labelnames)

        # Write the samples.
        for result in results:
            l = [result['metric'].get('__name__', '')] + result['value']
            for label in labelnames:
                l.append(result['metric'].get(label, ''))
            writer.writerow(l)



In [3]:
import csv
import requests
from datetime import datetime, timedelta 
from time import mktime

def query_csv_timeseries(q,state, time, step, filename):

    #generate timestamp + query
    now = datetime.now()
    now_unix =  mktime(now.timetuple())

    response = requests.get('http://localhost:9090/api/v1/query_range',
            params= {'query': q, 'start': now_unix - 3600 * time, "end": now_unix, "step": step})
    results = response.json()['data']['result']
    # Build a list of all labelnames used.
    labelnames = set()
    for result in results:
        labelnames.update(result['metric'].keys())


    # Canonicalize
    labelnames = sorted(labelnames)
    # print(labelnames)
    # print(result["values"])

    # with open( q + "_" + state + '.csv', mode='a') as f:
    with open(filename + '.csv', mode='a') as f:
        writer = csv.writer(f)
        # Write the header,
        #writer.writerow(labelnames)

        #Write the samples.
        for result in results:
            l = [result['metric'].get('__name__', '')] + result['values']
            
            #Prune the first element in each list, which is just a unix timestamp
            vals = [x[1] for x in result['values']]
            identifier_string = ""
            for label in labelnames:
                identifier_string += result['metric'].get(label, '') + "&"
                #l.append(result['metric'].get(label, ''))
                #writer.writerow(l)
                #print(l , "lllllll")
            #for label in labelnames:
                
            writer.writerow([identifier_string[:-1]] + vals)

metric = "jvm_memory_bytes_used"
state = "testing_2"
time = 0.2
step = "5m"
#query_csv_timeseries(metric, state, time, step)

#CSV BUilding. Put identifying information into one complete string as first element in each row. Then prune the timestamps to get only the numbers and put those into the csv 
#Put eerything in identifying string

In [6]:
import json
def query_collection(time, step, source, filename):
    file = open(source)
    metrics = json.load(file)['data']

    for i in metrics:
        query_csv_timeseries(i, "testing", time, step, filename)

    

#query_collection(9, "1m", "metrics.json", "heavy_load_4hv4")
query_collection(7, "1m", "metrics.json", "heavy_load_1m_stacked")

In [7]:
import schedule
import time

def doImports():
    query_collection(5, "1m", "metrics.json", "heavy_load_dunno_but_its_been_a_while_ill_find_out_hehe")
    print("running")

schedule.every(1).hours.do(doImports)


Every 1 hour do doImports() (last run: [never], next run: 2022-12-11 00:15:25)

In [8]:
#query_csv('avg by (job, instance, mode) (rate(process_cpu_seconds_total[5m]))', 'test1 after setup')
from datetime import datetime, timedelta 
from time import mktime
now = datetime.now()
twhoursago = now - timedelta(hours=2)
now_unix = mktime(now.timetuple())
twhoursago_unix = mktime(twhoursago.timetuple())
twhoursago_unix_manual = now_unix - 7200
print(now_unix, twhoursago_unix, twhoursago_unix_manual)
# node_cpu_seconds_total&start=2022-09-07T00:00:00Z&end=2022-09-07T02:00:00Z&step=5m

1669892339.0 1669885139.0 1669885139.0


In [1]:
import json
import csv
with open('heavy_load_1m_stacked.csv') as obj:
    reader = csv.reader(obj)
    list_of_csv = list(reader)
    useless_metrics = list()
    delta_metrics = list()
    metrics = list()

    for mlist in list_of_csv:
        if(mlist == []):
            continue
        label = mlist.pop(0)
        equal = all(i == j for i, j in zip(mlist, mlist[1:]))
        if equal:
            useless_metrics.append(label)
            continue
        res = all(i <= j for i, j in zip(mlist, mlist[1:]))
        if res:
            delta_metrics.append(label)
            continue
        metrics.append(label)
    print(len(useless_metrics), len(delta_metrics), len(metrics))
    with open("sorted_metrics.json", "w", encoding="utf-8") as f:
        json.dump({
            "delta_metrics": delta_metrics,
            "metrics": metrics,
            "useless_metrics": useless_metrics
        }, f, ensure_ascii=False, indent=4)


976 239 380
